In [0]:
dbutils.widgets.text("silver_path", "")
dbutils.widgets.text("gold_path", "")

silver_path = dbutils.widgets.get("silver_path")
gold_path = dbutils.widgets.get("gold_path")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
#from pyspark.sql.utils import AnalysisException

# spark.conf.set("fs.azure.account.key.adlsairqualitypoc.dfs.core.windows.net", "wyOLWhyzy7LbDaqB/GYL602VZsO98fuB5Elr6qUQBHV2uswoPxlHEfTyYS1bTRvKYQD2s/lrk2Uk+AStSn6bNg==")

# silver_path = "abfs://silver@adlsairqualitypoc.dfs.core.windows.net/aqi"
# gold_path   = "abfs://gold@adlsairqualitypoc.dfs.core.windows.net/aqi"

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

print("Starting Gold processing...")

In [0]:
silver_df = spark.read.format("delta").load(silver_path)

In [0]:
silver_df = (
    silver_df
    .withColumn("date", to_date("event_ts"))
    .withColumn("hour_ts", date_trunc("hour", col("event_ts")))
)

In [0]:
dim_location_path = f"{gold_path}/dim_location"

dim_location = (
    silver_df
    .select("country", "state", "city", "station")
    .distinct()
    .withColumn("location_key", monotonically_increasing_id())
)

if not DeltaTable.isDeltaTable(spark, dim_location_path):
    dim_location.write.format("delta").save(dim_location_path)
else:
    dim_location.write.format("delta").mode("overwrite").save(dim_location_path)

In [0]:
dim_pollutant_path = f"{gold_path}/dim_pollutant"

dim_pollutant = (
    silver_df
    .select("pollutant_id")
    .distinct()
    .withColumn("pollutant_key", monotonically_increasing_id())
)

if not DeltaTable.isDeltaTable(spark, dim_pollutant_path):
    dim_pollutant.write.format("delta").save(dim_pollutant_path)
else:
    dim_pollutant.write.format("delta").mode("overwrite").save(dim_pollutant_path)

In [0]:
dim_date_path = f"{gold_path}/dim_date"

dim_date = (
    silver_df
    .select("date")
    .distinct()
    .withColumn("year", year("date"))
    .withColumn("month", month("date"))
    .withColumn("day", dayofmonth("date"))
    .withColumn("date_key", col("date").cast("string"))
)

if not DeltaTable.isDeltaTable(spark, dim_date_path):
    dim_date.write.format("delta").save(dim_date_path)
else:
    dim_date.write.format("delta").mode("overwrite").save(dim_date_path)

In [0]:
fact_path = f"{gold_path}/fact_air_quality"

dim_location_df = spark.read.format("delta").load(dim_location_path)
dim_pollutant_df = spark.read.format("delta").load(dim_pollutant_path)

fact_df = (
    silver_df
    .join(dim_location_df, ["country","state","city","station"])
    .join(dim_pollutant_df, ["pollutant_id"])
    .select(
        "location_key",
        "pollutant_key",
        "event_ts",
        "date",
        "hour_ts",
        "pollutant_avg",
        "is_outlier",
        "ingestion_ts"
    )
)

if not DeltaTable.isDeltaTable(spark, fact_path):
    fact_df.write.format("delta").save(fact_path)
else:
    delta_table = DeltaTable.forPath(spark, fact_path)

    merge_condition = """
        t.location_key = s.location_key AND
        t.pollutant_key = s.pollutant_key AND
        t.event_ts = s.event_ts
    """

    (delta_table.alias("t")
        .merge(fact_df.alias("s"), merge_condition)
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

print("Fact table updated.")

In [0]:
agg_daily_state_path = f"{gold_path}/agg_daily_state"

fact_df_full = spark.read.format("delta").load(fact_path)

affected_dates = fact_df_full.select("date").distinct()
affected_dates.createOrReplaceTempView("affected_dates")

daily_state = (
    fact_df_full
    .groupBy("date")
    .agg(
        avg("pollutant_avg").alias("avg_aqi"),
        count("*").alias("reading_count")
    )
)

if not DeltaTable.isDeltaTable(spark, agg_daily_state_path):
    daily_state.write.format("delta").save(agg_daily_state_path)
else:
    spark.sql(f"""
        DELETE FROM delta.`{agg_daily_state_path}`
        WHERE date IN (SELECT date FROM affected_dates)
    """)
    daily_state.write.format("delta").mode("append").save(agg_daily_state_path)

print("Daily state aggregation updated.")

In [0]:
agg_daily_city_path = f"{gold_path}/agg_daily_city"

daily_city = (
    fact_df_full
    .groupBy("date", "location_key")
    .agg(
        avg("pollutant_avg").alias("avg_aqi"),
        count("*").alias("reading_count")
    )
)

if not DeltaTable.isDeltaTable(spark, agg_daily_city_path):
    daily_city.write.format("delta").save(agg_daily_city_path)
else:
    spark.sql(f"""
        DELETE FROM delta.`{agg_daily_city_path}`
        WHERE date IN (SELECT date FROM affected_dates)
    """)
    daily_city.write.format("delta").mode("append").save(agg_daily_city_path)

print("Daily city aggregation updated.")

print("Gold production processing complete.")

In [0]:
#threshold breach
threshold_breach = (
    silver_df
    .filter(col("pollutant_avg") > 200)
    .select(
        "state", "city", "station",
        "pollutant_id", "event_ts",
        "pollutant_avg"
    )
)

threshold_breach.write.format("delta").mode("overwrite") \
    .save(f"{gold_path}/fact_threshold_breach")

print("Threshold breach table updated.")

print("Gold processing completed.")

In [0]:
# spark.read.format("delta").load(fact_path).display(5)
# spark.read.format("delta").load(hourly_path).display(5)
# spark.read.format("delta").load(daily_path).display(5)

In [0]:
# %sql
# CREATE DATABASE IF NOT EXISTS aqi_gold;

# CREATE TABLE IF NOT EXISTS aqi_gold.fact_air_quality
# USING DELTA
# LOCATION 'abfs://gold@adlsairqualitypoc.dfs.core.windows.net/aqi/fact_air_quality';
